<h1><i> Final Capstone Project of Coursera </i></h1>

<h3> 1. Description problem </h3>

<p> In this final project we will help to an Bank to evaluate where can open in New York. </br>
Recently the bank is operating in Toronto. For the Executive Directory it's important know how the City of New York is clustering, looking for the principal businnes are exits in Manhattan.</p>

<p> Based on the New York Neighborhoods, we have to explore and find some similarys Neighbourhoods, compared with Toronto. </p>

<h3> 2. About the Data </h3>

<h4> 2.1 New York Dataset </h4>

<p> First, we will download the Neighborhoods dataset from <a href="https://geo.nyu.edu/catalog/nyu_2451_34572">New York: A City of Neighborhoods.</a></p>

In [1]:
import json
import pandas as pd

!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
    
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)   

neighborhoods_data = newyork_data['features']
df_n = pd.DataFrame()

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    df_n = df_n.append({'Borough': borough,
                        'Neighborhood': neighborhood_name,
                        'Latitude': neighborhood_lat,
                        'Longitude': neighborhood_lon,
                        'City': 'New York'
                       }, ignore_index=True)

df_n = df_n[['Borough','Neighborhood','Latitude','Longitude','City']]    
df_n.head()

,Borough,Neighborhood,Latitude,Longitude,City
0,Bronx,Wakefield,40.894705,-73.847201,New York
1,Bronx,Co-op City,40.874294,-73.829939,New York
2,Bronx,Eastchester,40.887556,-73.827806,New York
3,Bronx,Fieldston,40.895437,-73.905643,New York
4,Bronx,Riverdale,40.890834,-73.912585,New York


<p> Then we put the data on a Folium Map to visualize the New York Neighborhoods. </p>

In [2]:
import folium
from geopy.geocoders import Nominatim

manhattan_data = df_n[df_n['Borough'] == 'Manhattan'].reset_index(drop=True)
address = 'Manhattan, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudeNY = location.latitude
longitudeNY = location.longitude

map_manhattan = folium.Map(location=[latitudeNY, longitudeNY], zoom_start=11)

for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

<h4> 2.2 Toronto Dataset </h4>

<p> In second place, we need the neighbourhoods from Toronto, so for that we are going to download the postal code of each neighbourhood from Wikipedia through Web Scraping using BeautifulSoup libraries.</p>

In [3]:
import requests
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup

postal_codes = []

req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(req.text, "html.parser")

postal_table = soup.find('table',{"class":"wikitable sortable"})

for row in postal_table.find_all('tr'):
    cols = row.find_all('td')
    if len(cols) == 3:
        postal_codes.append((cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip(),'Toronto'))

df_t = pd.DataFrame(postal_codes)
df_t.columns = ['Postcode', 'Borough','Neighbourhood','City']
df_t = df_t[df_t.Borough != 'Not assigned']
df_t = df_t.rename(columns={'Postcode': 'PostalCode'})
df_t.loc[df_t['Neighbourhood'] == "Not assigned", 'Neighbourhood'] = df_t['Borough']
df_t.head()

,PostalCode,Borough,Neighbourhood,City
2,M3A,North York,Parkwoods,Toronto
3,M4A,North York,Victoria Village,Toronto
4,M5A,Downtown Toronto,Harbourfront,Toronto
5,M5A,Downtown Toronto,Regent Park,Toronto
6,M6A,North York,Lawrence Heights,Toronto


<p> Then we JOIN the Toronto dataset with Geospatial data obtained from Cognitive Class.</p>

In [4]:
!wget -q -O 'canada_data.csv' https://cocl.us/Geospatial_data

df_coordinates = pd.read_csv("canada_data.csv") 
df_coordinates = df_coordinates.rename(columns={'Postal Code':'PostalCode'})
df_coordinates = pd.merge(df_t, df_coordinates, on='PostalCode', how='inner')
df_coordinates = df_coordinates[['Borough','Neighbourhood','Latitude','Longitude','City']]   
df_coordinates = df_coordinates.rename(columns={'Neighbourhood':'Neighborhood'})

toronto_data = df_coordinates[df_coordinates['Borough'].str.contains('Toronto')]
df_t  = toronto_data
df_t = df_t.reset_index(drop=True)
df_t.head()

,Borough,Neighborhood,Latitude,Longitude,City
0,Downtown Toronto,Harbourfront,43.654260,-79.360636,Toronto
1,Downtown Toronto,Regent Park,43.654260,-79.360636,Toronto
2,Downtown Toronto,Ryerson,43.657162,-79.378937,Toronto
3,Downtown Toronto,Garden District,43.657162,-79.378937,Toronto
4,Downtown Toronto,St. James Town,43.651494,-79.375418,Toronto


<p> And same as New York, we put the data on a Folium Map to visualize the Neighborhoods in Toronto. </p>

In [5]:
import folium 
from geopy.geocoders import Nominatim 

address = 'Toronto, ON'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitudeTO = location.latitude
longitudeTO = location.longitude

map_canada = folium.Map(location=[latitudeTO, longitudeTO], zoom_start=11)

for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

<h4> 2.3 Brief summary </h4>

In [6]:
print('The data set from New York has {} rows.'.format(df_n.count().unique()))
print('The data set from Toronto has {} rows.'.format(df_t.count().unique()))

print('New York has {} unique Borough.'.format(len(df_n['Borough'].unique())))
print('Toronto has {} unique Borough.'.format(len(df_t['Borough'].unique())))

The data set from New York has [306] rows.
The data set from Toronto has [74] rows.
New York has 5 unique Borough.
Toronto has 4 unique Borough.


<h4> 2.4 Mix both DataFrame in a single one </h4>

In [7]:
neighborhoods = pd.concat([df_n, df_t])
neighborhoods = neighborhoods.reset_index(drop=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude,City
0,Bronx,Wakefield,40.894705,-73.847201,New York
1,Bronx,Co-op City,40.874294,-73.829939,New York
2,Bronx,Eastchester,40.887556,-73.827806,New York
3,Bronx,Fieldston,40.895437,-73.905643,New York
4,Bronx,Riverdale,40.890834,-73.912585,New York


<h3> 3. Methodology </h3>

<h4> 3.1 Connecting to API Foursquare </h4>

<p> We will use the Foursquare API to get the nearby venues and k-means clustering algorithm to analyze the Neighbourhood.</p>

In [53]:
#CLIENT_ID = #yourClientID
#CLIENT_SECRET = #yourClientSecret

VERSION = '20190315'
LIMIT = 100
radius = 500

<p> Then define an function to download the nearby venues from the API Foursquare. </p>

In [9]:
def getNearbyVenues(names, latitudes, longitudes, city, radius):

    venues_list = []

    for name, lat, lng, city in zip(names, latitudes, longitudes, city):

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                                    CLIENT_ID, 
                                    CLIENT_SECRET, 
                                    VERSION, 
                                    lat,
                                    lng,
                                    radius, 
                                    LIMIT)

        results = requests.get(url).json()    
        venues = results['response']['groups'][0]['items']

        venues_list.append([(
            name, 
            lat, 
            lng,
            city,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venues])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                      'Neighborhood Latitude', 
                      'Neighborhood Longitude', 
                      'Neighborhood City', 
                      'Venue', 
                      'Venue Latitude', 
                      'Venue Longitude', 
                      'Venue Category']
    #error Groups tiene relación con la cuota excedida con la API de Foursquare
    return(nearby_venues)

<p> We called the function to obtain de nearby venues from the DataFrame Neighborhoods. </p>

<h4> 3.2 Download data from Foursquare </h4>

In [10]:
neighborhoods_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'],
                                   city=neighborhoods['City'],    
                                   radius=500
                                  )

<p> Below show the existing venues from each Neighborhood with a Radius of 500. </p>

In [11]:
neighborhoods_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Neighborhood City,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Adelaide,100,100,100,100,100,100,100
Allerton,30,30,30,30,30,30,30
Annadale,11,11,11,11,11,11,11
Arden Heights,5,5,5,5,5,5,5
Arlington,8,8,8,8,8,8,8


<p> Now look for how many uniques categories of venues. </p>

In [12]:
print('There are {} uniques categories.'.format(len(neighborhoods_venues['Venue Category'].unique())))

There are 455 uniques categories.


<h3> 4. Analyze Toronto and New York Neighborhoods </h3>

In [13]:
# one hot encoding
neighborhoods_onehot = pd.get_dummies(neighborhoods_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhoods_onehot['Neighborhood'] = neighborhoods_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neighborhoods_onehot.columns[-1]] + list(neighborhoods_onehot.columns[:-1])
neighborhoods_onehot = neighborhoods_onehot[fixed_columns]

neighborhoods_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Warehouse Store,Waste Facility,Watch Shop,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<p> This is our new dataframe with the neighborhoods. </p>

In [14]:
neighborhoods_onehot.shape

(13561, 455)

<p> Analyze the mean of the frequency of occurrence of each category grouped by Neighborhood. </p>

In [15]:
neighborhoods_grouped = neighborhoods_onehot.groupby('Neighborhood').mean().reset_index()
neighborhoods_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Warehouse Store,Waste Facility,Watch Shop,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adelaide,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01
1,Allerton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
2,Annadale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
3,Arden Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00
4,Arlington,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00


<h4> 4.1 Search for Top 5 venues in each neighborhood </h4>

In [16]:
num_top_venues = 5

for hood in neighborhoods_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = neighborhoods_grouped[neighborhoods_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
             venue  freq
0      Coffee Shop  0.06
1             Café  0.05
2              Bar  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Allerton----
                venue  freq
0       Deli / Bodega  0.10
1         Pizza Place  0.10
2  Chinese Restaurant  0.07
3         Supermarket  0.07
4        Liquor Store  0.07


----Annadale----
           venue  freq
0    Pizza Place  0.18
1            Pub  0.09
2          Diner  0.09
3  Train Station  0.09
4     Restaurant  0.09


----Arden Heights----
           venue  freq
0       Bus Stop   0.2
1       Pharmacy   0.2
2    Pizza Place   0.2
3  Deli / Bodega   0.2
4    Coffee Shop   0.2


----Arlington----
           venue  freq
0  Grocery Store  0.12
1    Coffee Shop  0.12
2           Cave  0.12
3       Bus Stop  0.12
4  Deli / Bodega  0.12


----Arrochar----
                   venue  freq
0               Bus Stop  0.16
1     Italian Restaurant  0.11
2          Deli / Bodega  0.11
3            Supermarket  

<h4> 4.2 Define a function to sort the venues in descending order </h4>

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<p> Now we are going to look for the top 10 venues for each neighborhood. </p>

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neighborhoods_grouped['Neighborhood']

for ind in np.arange(neighborhoods_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhoods_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Bar,Thai Restaurant,Steakhouse,Gym,Burger Joint,Hotel,Bakery,Asian Restaurant
1,Allerton,Deli / Bodega,Pizza Place,Chinese Restaurant,Liquor Store,Supermarket,Spa,Martial Arts Dojo,Discount Store,Bakery,Electronics Store
2,Annadale,Pizza Place,Sports Bar,Diner,Food,Train Station,Pub,Restaurant,Sushi Restaurant,Dance Studio,Liquor Store
3,Arden Heights,Pharmacy,Bus Stop,Pizza Place,Deli / Bodega,Coffee Shop,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space
4,Arlington,Deli / Bodega,Bus Stop,Intersection,Cave,Boat or Ferry,Coffee Shop,American Restaurant,Grocery Store,Factory,Farm


<h3> 5. Cluster Neighborhoods </h3>

<p> Then we run k-means to cluster into 10 clusters the neighborhood. </p>

In [19]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 10

neighborhoods_grouped_clustering = neighborhoods_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhoods_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 4, 3, 3, 0, 3, 3, 0, 4, 0], dtype=int32)

<p> Now create an dataframe that includes the cluster with the top 10 venues for each neighborhood. </p>

In [20]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

<p> We check the new dataframe. </p>

In [21]:
neighborhoods_merged = neighborhoods
neighborhoods_merged.head()

,Borough,Neighborhood,Latitude,Longitude,City
0,Bronx,Wakefield,40.894705,-73.847201,New York
1,Bronx,Co-op City,40.874294,-73.829939,New York
2,Bronx,Eastchester,40.887556,-73.827806,New York
3,Bronx,Fieldston,40.895437,-73.905643,New York
4,Bronx,Riverdale,40.890834,-73.912585,New York


<p> Merge the new dataframe with the venues sorted ones.</p>

In [22]:
neighborhoods_merged = neighborhoods_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

<p> Then create two data frame to separate the neighborhoods from New York and Toronto. </p>

In [23]:
newyork_merged = neighborhoods_merged[neighborhoods_merged['City'].str.contains('New York') == True]
toronto_merged = neighborhoods_merged[neighborhoods_merged['City'].str.contains('Toronto') == True]

In [24]:
newyork_merged = newyork_merged.rename(columns={'Cluster Labels':'ClusterLabels'})
toronto_merged = toronto_merged.rename(columns={'Cluster Labels':'ClusterLabels'})
newyork_merged = newyork_merged[newyork_merged['ClusterLabels'].notnull()]
toronto_merged = toronto_merged[toronto_merged['ClusterLabels'].notnull()]

<p> Put the data cluster on a Folium Map to visualize. </p>

In [38]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

# create map
map_clusters_ny = folium.Map(location=[latitudeNY, longitudeNY], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newyork_merged['Latitude'], newyork_merged['Longitude'], newyork_merged['Neighborhood'], newyork_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters_ny)
       
map_clusters_ny

<p> Repeat the same with Toronto cluster. </p>

In [40]:
# create map of Toronto
map_clusters_to = folium.Map(location=[latitudeTO, longitudeTO], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters_to)
       
map_clusters_to

<h3> 6. Examine the cluster </h3>

In [41]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 0,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,New York,0,Playground,Plaza,Women's Store,Farmers Market,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
4,Riverdale,New York,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Park,Plaza,Playground,Pizza Place,Grocery Store
6,Marble Hill,New York,0,Discount Store,Coffee Shop,Yoga Studio,Pizza Place,Steakhouse,Spa,Supplement Shop,Big Box Store,Shoe Store,Tennis Stadium
12,City Island,New York,0,Harbor / Marina,Seafood Restaurant,Thrift / Vintage Store,Italian Restaurant,American Restaurant,Park,Smoke Shop,Art Gallery,History Museum,Bank
22,Port Morris,New York,0,Food Truck,Music Venue,Donut Shop,Distillery,Spanish Restaurant,Furniture / Home Store,Latin American Restaurant,Restaurant,Cupcake Shop,Bookstore
24,Hunts Point,New York,0,BBQ Joint,Waste Facility,Farmers Market,Gourmet Shop,Spanish Restaurant,Restaurant,Café,Seafood Restaurant,Juice Bar,Bank
27,Clason Point,New York,0,Park,Bus Stop,Home Service,Boat or Ferry,Grocery Store,Pool,Scenic Lookout,South American Restaurant,English Restaurant,Ethiopian Restaurant
29,Country Club,New York,0,Sandwich Place,Spa,Fried Chicken Joint,Playground,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space
46,Bay Ridge,New York,0,Italian Restaurant,Spa,Pizza Place,Bar,Greek Restaurant,Bagel Shop,American Restaurant,Playground,Cosmetics Shop,Grocery Store
47,Bensonhurst,New York,0,Chinese Restaurant,Italian Restaurant,Park,Ice Cream Shop,Sushi Restaurant,Donut Shop,Pharmacy,Pet Store,Dumpling Restaurant,Liquor Store


In [42]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 1,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
202,Grymes Hill,New York,1,Dog Run,Women's Store,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory


In [45]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 2,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
192,Somerville,New York,2,Park,Women's Store,Fried Chicken Joint,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
203,Todt Hill,New York,2,Park,Women's Store,Fried Chicken Joint,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor


In [46]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 3,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Kingsbridge,New York,3,Pizza Place,Bar,Sandwich Place,Supermarket,Discount Store,Bakery,Mexican Restaurant,Latin American Restaurant,Park,Deli / Bodega
7,Woodlawn,New York,3,Food & Drink Shop,Pizza Place,Playground,Deli / Bodega,Bar,Italian Restaurant,Train Station,Indian Restaurant,Moving Target,Food Truck
8,Norwood,New York,3,Park,Pizza Place,Chinese Restaurant,Pharmacy,Deli / Bodega,Bank,American Restaurant,Mexican Restaurant,Sandwich Place,Coffee Shop
10,Baychester,New York,3,Electronics Store,American Restaurant,Bank,Gym / Fitness Center,Men's Store,Mexican Restaurant,Pet Store,Donut Shop,Sandwich Place,Fast Food Restaurant
11,Pelham Parkway,New York,3,Italian Restaurant,Frozen Yogurt Shop,Pizza Place,Bus Station,Plaza,Thai Restaurant,Mexican Restaurant,Chinese Restaurant,Donut Shop,Eye Doctor
17,East Tremont,New York,3,Pizza Place,Breakfast Spot,Discount Store,Shoe Store,Chinese Restaurant,Lounge,Mexican Restaurant,Paella Restaurant,Mobile Phone Shop,Donut Shop
20,Melrose,New York,3,Pizza Place,Pharmacy,Supermarket,Gym / Fitness Center,Discount Store,Sandwich Place,Diner,Donut Shop,Art Gallery,Platform
21,Mott Haven,New York,3,Pizza Place,Gym,Spanish Restaurant,Chinese Restaurant,Fish & Chips Shop,Baseball Field,Peruvian Restaurant,Grocery Store,Pharmacy,Mobile Phone Shop
28,Throgs Neck,New York,3,Deli / Bodega,Juice Bar,Pizza Place,American Restaurant,Asian Restaurant,Sports Bar,Italian Restaurant,Bar,Coffee Shop,Exhibit
30,Parkchester,New York,3,Supermarket,Pizza Place,American Restaurant,Asian Restaurant,Bank,Women's Store,Mexican Restaurant,Shoe Store,Shipping Store,Sandwich Place


In [47]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 4,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,New York,4,Food Truck,Ice Cream Shop,Caribbean Restaurant,Donut Shop,Sandwich Place,Dessert Shop,Laundromat,Pharmacy,Factory,Food Stand
1,Co-op City,New York,4,Bus Station,Baseball Field,Chinese Restaurant,Grocery Store,Pharmacy,Liquor Store,Park,Pizza Place,Discount Store,Fast Food Restaurant
2,Eastchester,New York,4,Caribbean Restaurant,Deli / Bodega,Diner,Bus Stop,Bus Station,Convenience Store,Metro Station,Cosmetics Shop,Platform,Pizza Place
9,Williamsbridge,New York,4,Nightclub,Supermarket,Bar,Soup Place,Caribbean Restaurant,Farmers Market,English Restaurant,Ethiopian Restaurant,Event Service,Event Space
13,Bedford Park,New York,4,Deli / Bodega,Diner,Mexican Restaurant,Pizza Place,Supermarket,Sandwich Place,Chinese Restaurant,Fried Chicken Joint,Spanish Restaurant,Laundromat
14,University Heights,New York,4,Pizza Place,Food Truck,Bus Station,Bank,Bakery,Chinese Restaurant,Pharmacy,Donut Shop,Sandwich Place,History Museum
15,Morris Heights,New York,4,Food Truck,Grocery Store,Latin American Restaurant,Pizza Place,Pharmacy,Bank,Buffet,Bus Station,Burrito Place,Spanish Restaurant
16,Fordham,New York,4,Shoe Store,Mobile Phone Shop,Donut Shop,Fast Food Restaurant,Pizza Place,Spanish Restaurant,Gym / Fitness Center,Pharmacy,Supplement Shop,Bank
18,West Farms,New York,4,Bus Station,Latin American Restaurant,Basketball Court,Metro Station,Park,Lounge,Donut Shop,Coffee Shop,Sandwich Place,Playground
19,High Bridge,New York,4,Chinese Restaurant,Pharmacy,Pizza Place,Supermarket,Deli / Bodega,Sandwich Place,Bus Station,Gym,Park,Market


In [48]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 5,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
193,Brookville,New York,5,Deli / Bodega,Women's Store,Field,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory


In [49]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 6,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
77,Manhattan Beach,New York,6,Bus Stop,Café,Beach,Harbor / Marina,Playground,Food,Ice Cream Shop,Sandwich Place,Pizza Place,Eye Doctor
85,Sea Gate,New York,6,Bus Station,Optical Shop,Spa,Business Service,Beach,Field,Ethiopian Restaurant,Event Service,Event Space,Exhibit
172,Breezy Point,New York,6,Monument / Landmark,Beach,Construction & Landscaping,Board Shop,Trail,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
178,Rockaway Beach,New York,6,Beach,Ice Cream Shop,Pizza Place,Brazilian Restaurant,Seafood Restaurant,Latin American Restaurant,Arepa Restaurant,BBQ Joint,Bagel Shop,Bus Station
179,Neponsit,New York,6,Beach,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant
190,Belle Harbor,New York,6,Beach,Spa,Deli / Bodega,Italian Restaurant,Chinese Restaurant,Mexican Restaurant,Pharmacy,Donut Shop,Bagel Shop,Bus Stop
198,New Brighton,New York,6,Bus Stop,Deli / Bodega,Park,Bowling Alley,Playground,Home Service,Discount Store,Convenience Store,Food & Drink Shop,Food
204,South Beach,New York,6,Pier,Bus Stop,Beach,Athletics & Sports,Deli / Bodega,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
206,Mariner's Harbor,New York,6,Italian Restaurant,Deli / Bodega,Nightlife Spot,Other Repair Shop,Bus Stop,Field,Ethiopian Restaurant,Event Service,Event Space,Exhibit
217,Tottenville,New York,6,Italian Restaurant,Cosmetics Shop,Thrift / Vintage Store,Mexican Restaurant,Deli / Bodega,Bus Stop,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Event Space


In [50]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 7,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
207,Port Ivory,New York,7,Bar,Women's Store,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory


In [51]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 8,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
257,Howland Hook,New York,8,Pier,Women's Store,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor


In [52]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 9,neighborhoods_merged.columns[[1] + [4] + list(range(5, neighborhoods_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
212,Oakwood,New York,9,Bus Stop,Bus Station,Bar,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant
224,Park Hill,New York,9,Bus Stop,Athletics & Sports,Gym / Fitness Center,Coffee Shop,Hotel,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space
226,Graniteville,New York,9,Bus Stop,Food Truck,Grocery Store,Women's Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
256,Randall Manor,New York,9,Bus Stop,Bagel Shop,Playground,Women's Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
305,Fox Hills,New York,9,Bus Stop,Sandwich Place,Grocery Store,Deli / Bodega,Women's Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
